<a href="https://colab.research.google.com/github/ConnBreathnach/LiloGuesser/blob/main/Chess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

VOCAB_SIZE = 10863
BATCH_SIZE=1024
data = pd.read_csv("/content/drive/MyDrive/ChessData/train_games.csv")
data.dropna(inplace=True)

data['white_min_elo'] = pd.Categorical(data['white_min_elo'])
data['white_min_elo'] = data.white_min_elo.cat.codes

features = data.pop('moves').astype("string")
labels = data.pop('white_min_elo')

dataset = tf.data.Dataset.from_tensor_slices((features, labels)).batch(BATCH_SIZE)


test_data = pd.read_csv("/content/drive/MyDrive/ChessData/test_games.csv")
test_data.dropna(inplace=True)
test_data['white_min_elo'] = pd.Categorical(test_data['white_min_elo'])
test_data['white_min_elo'] = test_data.white_min_elo.cat.codes
test_features = test_data.pop('moves').astype("string")
test_labels = test_data.pop('white_min_elo')
test_dataset = tf.data.Dataset.from_tensor_slices((test_features,test_labels)).batch(BATCH_SIZE)


In [ ]:
vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=150
)

In [ ]:
vectorize_layer.adapt(dataset.map(lambda text, labels: text))

In [ ]:
def get_model():
    model = tf.keras.Sequential(
        [
            vectorize_layer,
            tf.keras.layers.Embedding(
                input_dim=len(vectorize_layer.get_vocabulary()),
                output_dim=1024,
                mask_zero=True
            ),
            tf.keras.layers.Bidirectional(tf.keras.layers.GRU(1024)),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(21)
        ]
    )
    model.compile(optimizer="SGD", loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    return model


In [ ]:
  
sample_game = "e4 e5 bc4 nc6 qf3 a6 qxf7#"

In [ ]:
new_model = get_model()
# new_model.predict(np.array([sample_game]))
# new_model.summary()
new_history = new_model.fit(dataset, epochs=15, validation_data=test_dataset, validation_steps=30)

Epoch 1/15
879/879 [==============================] - 3669s 4s/step - loss: 2.4394 - accuracy: 0.1660 - val_loss: 2.3082 - val_accuracy: 0.1657
Epoch 2/15
879/879 [==============================] - 3665s 4s/step - loss: 2.2893 - accuracy: 0.1678 - val_loss: 2.2972 - val_accuracy: 0.1658
Epoch 3/15
879/879 [==============================] - 3676s 4s/step - loss: 2.2846 - accuracy: 0.1683 - val_loss: 2.2945 - val_accuracy: 0.1666
Epoch 4/15
879/879 [==============================] - 3673s 4s/step - loss: 2.2828 - accuracy: 0.1688 - val_loss: 2.2930 - val_accuracy: 0.1686
Epoch 5/15
879/879 [==============================] - 3680s 4s/step - loss: 2.2815 - accuracy: 0.1694 - val_loss: 2.2916 - val_accuracy: 0.1723
Epoch 6/15
122/879 [===>..........................] - ETA: 51:38 - loss: 2.2829 - accuracy: 0.1700